In [1]:
!pip install qiskit
!pip install qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 99.2 MB/s eta 0:00:00


In [2]:
# Deutsch–Jozsa Algorithm using Qiskit 2.x
# Compatible with Qiskit >= 2.0.0

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt


In [3]:
# ---------- ORACLES ----------
def oracle_constant(qc, ancilla, value=0):
    """Constant oracle: f(x)=0 or f(x)=1"""
    if value == 1:
        qc.x(ancilla)


def oracle_balanced_parity(qc, inputs, ancilla):
    """Balanced oracle: f(x) = x0 XOR x1 XOR ... XOR xn"""
    for q in inputs:
        qc.cx(q, ancilla)




In [4]:
# ---------- DEUTSCH–JOZSA CIRCUIT ----------
def deutsch_jozsa_circuit(n, oracle_func, *oracle_args):
    """
    n: number of input qubits
    oracle_func: oracle function to modify the circuit
    oracle_args: extra arguments for oracle
    """
    qreg = QuantumRegister(n + 1, "q")
    creg = ClassicalRegister(n, "c")
    qc = QuantumCircuit(qreg, creg)

    inputs = list(range(n))
    ancilla = n

    # Step 1: Initialize |0...0>|1>
    qc.x(ancilla)

    # Step 2: Apply Hadamard to all qubits
    qc.h(qreg)

    # Step 3: Oracle
    oracle_func(qc, *oracle_args)

    # Step 4: Apply Hadamard to input qubits
    for q in inputs:
        qc.h(q)

    # Step 5: Measure only input qubits
    qc.measure(inputs, creg)

    return qc


In [5]:
# ---------- EXECUTION ----------
def run_dj(qc):
    """Run Deutsch–Jozsa circuit on AerSimulator"""
    simulator = AerSimulator()
    tqc = transpile(qc, simulator)
    job = simulator.run(tqc, shots=1024)
    result = job.result()
    counts = result.get_counts()

    print("Measurement counts:", counts)
    plot_histogram(counts)
    plt.show()

    n = qc.num_clbits
    if counts.get("0" * n, 0) == 1024:
        print("✅ Function is CONSTANT")
    else:
        print("✅ Function is BALANCED")


In [6]:
# ---------- MAIN ----------
if __name__ == "__main__":
    n = 3  # number of input qubits

    print("\n=== Constant Oracle (f(x)=0) ===")
    qc_const = deutsch_jozsa_circuit(
        n, oracle_constant, n, 0
    )
    print(qc_const.draw(fold=-1))
    run_dj(qc_const)

    print("\n=== Balanced Oracle (Parity) ===")
    qc_balanced = deutsch_jozsa_circuit(
        n, oracle_balanced_parity, list(range(n)), n
    )
    print(qc_balanced.draw(fold=-1))
    run_dj(qc_balanced)



=== Constant Oracle (f(x)=0) ===
     ┌───┐┌───┐┌─┐      
q_0: ┤ H ├┤ H ├┤M├──────
     ├───┤├───┤└╥┘┌─┐   
q_1: ┤ H ├┤ H ├─╫─┤M├───
     ├───┤├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├┤ H ├─╫──╫─┤M├
     ├───┤├───┤ ║  ║ └╥┘
q_3: ┤ X ├┤ H ├─╫──╫──╫─
     └───┘└───┘ ║  ║  ║ 
c: 3/═══════════╩══╩══╩═
                0  1  2 
Measurement counts: {'000': 1024}
✅ Function is CONSTANT

=== Balanced Oracle (Parity) ===
     ┌───┐          ┌───┐     ┌─┐           
q_0: ┤ H ├───────■──┤ H ├─────┤M├───────────
     ├───┤       │  └───┘┌───┐└╥┘     ┌─┐   
q_1: ┤ H ├───────┼────■──┤ H ├─╫──────┤M├───
     ├───┤       │    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_2: ┤ H ├───────┼────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_3: ┤ X ├┤ H ├┤ X ├┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘└───┘└───┘ ║       ║  ║ 
c: 3/══════════════════════════╩═══════╩══╩═
                               0       1  2 
Measurement counts: {'111': 1024}
✅ Function is BALANCED


In [8]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit_aer.noise import NoiseModel, depolarizing_error, ReadoutError
from qiskit.quantum_info import Operator
import matplotlib.pyplot as plt


In [9]:
def oracle_constant(qc, ancilla, value=0):
    if value == 1:
        qc.x(ancilla)

def oracle_balanced_parity(qc, inputs, ancilla):
    for q in inputs:
        qc.cx(q, ancilla)


In [10]:
def oracle_balanced_custom(qc, inputs, ancilla):
    """
    Balanced Oracle:
    f(x) = x0 XOR (x1 AND x2)
    Requires at least 3 input qubits.
    """
    if len(inputs) < 3:
        raise ValueError("Need at least 3 qubits for this oracle.")

    # ancilla ^= x0
    qc.cx(inputs[0], ancilla)

    # ancilla ^= (x1 AND x2)
    qc.ccx(inputs[1], inputs[2], ancilla)


In [11]:
def deutsch_jozsa(n, oracle, *oracle_args):
    q = QuantumRegister(n+1, "q")
    c = ClassicalRegister(n, "c")
    qc = QuantumCircuit(q, c)

    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)
    qc.h(q)

    oracle(qc, *oracle_args)

    for i in inputs:
        qc.h(i)

    qc.measure(inputs, c)
    return qc


In [12]:
def run_dj(qc, shots=1024):
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    result = sim.run(tqc, shots=shots).result()
    counts = result.get_counts()
    plot_histogram(counts)
    plt.show()

    if counts.get("0" * qc.num_clbits, 0) == shots:
        print("Function is CONSTANT")
    else:
        print("Function is BALANCED")

    return counts


In [13]:
n = 3
inputs = list(range(n))
ancilla = n

qc_balanced_custom = deutsch_jozsa(n, oracle_balanced_custom, inputs, ancilla)
print(qc_balanced_custom.draw())
run_dj(qc_balanced_custom)


     ┌───┐          ┌───┐     ┌─┐      
q_0: ┤ H ├───────■──┤ H ├─────┤M├──────
     ├───┤       │  └───┘┌───┐└╥┘┌─┐   
q_1: ┤ H ├───────┼────■──┤ H ├─╫─┤M├───
     ├───┤       │    │  ├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├───────┼────■──┤ H ├─╫──╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐└───┘ ║  ║ └╥┘
q_3: ┤ X ├┤ H ├┤ X ├┤ X ├──────╫──╫──╫─
     └───┘└───┘└───┘└───┘      ║  ║  ║ 
c: 3/══════════════════════════╩══╩══╩═
                               0  1  2 
Function is BALANCED


{'001': 265, '101': 248, '011': 265, '111': 246}

In [14]:
for n in [2, 3, 4, 5]:
    try:
        inputs = list(range(n))
        ancilla = n
        qc = deutsch_jozsa(n, oracle_balanced_custom, inputs, ancilla)
        print(f"\nFor n={n}, circuit depth={qc.depth()}")
        run_dj(qc)
    except:
        print(f"Skipping n={n}: custom oracle requires n>=3.")


Skipping n=2: custom oracle requires n>=3.

For n=3, circuit depth=6
Function is BALANCED

For n=4, circuit depth=6
Function is BALANCED

For n=5, circuit depth=6
Function is BALANCED


In [15]:
def build_noise():
    nm = NoiseModel()
    nm.add_all_qubit_quantum_error(depolarizing_error(0.01, 1), ['x', 'h'])
    nm.add_all_qubit_quantum_error(depolarizing_error(0.05, 2), ['cx'])
    ro = ReadoutError([[0.9, 0.1], [0.1, 0.9]])
    nm.add_all_qubit_readout_error(ro)
    return nm

noise_model = build_noise()
sim_noisy = AerSimulator(noise_model=noise_model)

tqc = transpile(qc_balanced_custom, sim_noisy)
res = sim_noisy.run(tqc, shots=2048).result()
counts_noisy = res.get_counts()
plot_histogram(counts_noisy)
plt.show()

print("Noisy measurement:", counts_noisy)


Noisy measurement: {'100': 69, '110': 50, '000': 87, '010': 61, '101': 457, '011': 424, '111': 433, '001': 467}


In [26]:
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector, Pauli

sim_noisy_estimator = AerSimulator(noise_model=noise_model)

tqc = qc_balanced_custom.copy()
tqc.remove_final_measurements()

tqc_with_measurements = qc_balanced_custom.copy()

shots = 2048
result_noisy = sim_noisy_estimator.run(tqc_with_measurements, shots=shots).result()
counts_noisy = result_noisy.get_counts()

print("Noisy measurement counts:", counts_noisy)
n = qc_balanced_custom.num_clbits
total_shots = sum(counts_noisy.values())
count_zeros = counts_noisy.get("0"*n, 0)
count_others = total_shots - count_zeros

expectation_value = 0
for bitstring, count in counts_noisy.items():
    parity = sum(int(bit) for bit in bitstring)
    sign = (-1)**parity
    expectation_value += count * sign

expectation_value /= total_shots

print(f"Estimated expectation value of Z^{n}: {expectation_value}")
if expectation_value > 0:
    print("Inferred: Function is likely CONSTANT (due to noise, might not be exact 1)")
else:
    print("Inferred: Function is likely BALANCED (due to noise, might not be exact -1)")

Noisy measurement counts: {'110': 56, '001': 472, '101': 429, '000': 112, '010': 64, '111': 433, '011': 426, '100': 56}
Estimated expectation value of Z^3: -0.0009765625
Inferred: Function is likely BALANCED (due to noise, might not be exact -1)


In [19]:
from qiskit.quantum_info import Operator

q = QuantumRegister(4, "q")
qc_oracle = QuantumCircuit(q)

oracle_balanced_custom(qc_oracle, [0,1,2], 3)
unitary = Operator(qc_oracle).data

print("Oracle Decomposition:")
print(qc_oracle.to_gate().definition)
print("\nUnitary Matrix (size: {}x{}):".format(len(unitary), len(unitary)))
print(unitary)


Oracle Decomposition:
               
q_0: ──■───────
       │       
q_1: ──┼────■──
       │    │  
q_2: ──┼────■──
     ┌─┴─┐┌─┴─┐
q_3: ┤ X ├┤ X ├
     └───┘└───┘

Unitary Matrix (size: 16x16):
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j